In [ ]:
!pip3 install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import string
import wikipediaapi
import re

##**Data Collection**

**Get data from Wikipedia**

In [ ]:
def wiki_data(topic="programming language", language='en'):

  wiki_wiki = wikipediaapi.Wikipedia(
          language= language,
          extract_format=wikipediaapi.ExtractFormat.WIKI
  )

  p_wiki = wiki_wiki.page(topic)
  data = p_wiki.text.strip()

  return data

In [ ]:
def loading_data(clasess=['ai', 'sport'], topics = [['Machine Learning', 'Deep Learning', 'Reinforcement Learning', 'Artificial Intelligence', 'chatgpt'], ['Football', 'Mohamed Salah', 'Tennis', 'Basketball', 'C.Ronaldo'], ['Tensorflow', 'Messi']]):
  data = {'Doc':[], 'Class':[]}

  #Training docs
  for i, c in enumerate(clasess):
    for topic in topics[i]:
      data['Doc'].append(wiki_data(topic = topic))
      data['Class'].append(c)

  #Testing docs
  for topic in topics[-1]:
    data['Doc'].append(wiki_data(topic = topic))
    data['Class'].append('unknown')

  df = pd.DataFrame (data, columns = data.keys())

  return df


df = loading_data()

In [ ]:
df

,Doc,Class
0,Machine learning (ML) is a field devoted to un...,ai
1,Deep learning is part of a broader family of m...,ai
2,Reinforcement learning (RL) is an area of mach...,ai
3,Artificial intelligence (AI) is intelligence—p...,ai
4,ChatGPT is an artificial intelligence (AI) cha...,ai
5,Football is a family of team sports that invol...,sport
6,Mohamed Salah Hamed Mahrous Ghaly (Arabic: محم...,sport
7,Tennis is a racket sport that is played either...,sport
8,"Basketball is a team sport in which two teams,...",sport
9,Cristiano Ronaldo dos Santos Aveiro (Portugue...,sport


In [ ]:
train = df.iloc[:10]
test = df.iloc[10:]

X_train = np.array(train['Doc'])
y_train = np.array(train['Class'])

X_test = np.array(test['Doc'])

In [ ]:
def class_prob(doc_label):
  labels = np.unique(doc_label)
  probs = []
  for label in labels:
    probs.append(len(doc_label[doc_label == label])/len(doc_label))

  return np.array(probs)

probs = class_prob(y_train)

##**TEXT-PREPROCESSING**

In [ ]:
def remove_pg_num(text):
    text = re.sub('[[]\d+[]]|[[]\d[]]', '', text)
    return text


In [ ]:
def remove_extra_spaces(text):
  text = re.sub('\s+', ' ', text)
  return text

In [ ]:
def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

In [ ]:
def remove_ar(text):
    text = re.sub('[أ-ي]', '', text)
    return text

In [ ]:
def remove_num(text):
    text = re.sub('[0-9]', '', text)
    return text

In [ ]:
def remove_mails(text):
    text = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", text)
    return text

In [ ]:
# def remove_dot(text):
#     text = re.sub(r'\.+', ".", text)
#     text = re.sub(r'\,+', ",", text)
#     return text

In [ ]:
def remove_punctuations(text):
    punctuations = '!"#$%&\()*/:;<=>?@[\\]^_`{|}~-.,–'
    return text.translate(str.maketrans('', '', punctuations))

In [ ]:
def remove_single_quotes(text):
    text = re.sub("'", '', text)
    return text

**TEXT-PREPROCESSING (FUNCTION)**

In [ ]:
def text_preprocessing(docs):

  if type(docs) == type(np.str_('mohamed')):
    doc = docs
    doc = doc.lower()
    doc = remove_pg_num(doc)
    doc = remove_url(doc)
    doc = remove_ar(doc)
    doc = remove_mails(doc)
    # doc = contractions.fix(data)
    doc = remove_punctuations(doc)
    doc = remove_num(doc)
    # doc = remove_dot(doc)
    doc = remove_single_quotes(doc)
    doc = remove_extra_spaces(doc)

    return doc[:]

  else:
    data=[]
    for doc in docs:
      doc = doc.lower()
      doc = remove_pg_num(doc)
      doc = remove_url(doc)
      doc = remove_ar(doc)
      doc = remove_mails(doc)
      # doc = contractions.fix(data)
      doc = remove_punctuations(doc)
      doc = remove_num(doc)
      # doc = remove_dot(doc)
      doc = remove_single_quotes(doc)
      doc = remove_extra_spaces(doc)

      data.append(doc[:])

    return np.array(data)



X_train = text_preprocessing(X_train)

X_test = text_preprocessing(X_test)

In [ ]:
def getTokens(doc):
  from nltk.stem import PorterStemmer
  ps = PorterStemmer()

  doc = doc.split()
  stop_words= ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'm', 'also',  'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'x', 'y', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
  doc = [ps.stem(x) for x in doc if x not in stop_words]

  return doc

#Naive Bayes

In [ ]:
def conditional_prob(X_train, y_train):

  X_train = text_preprocessing(X_train)
  labels = np.unique(y_train)
  c_words = []
  for label in labels:
    c_words.append(getTokens(" ".join(X_train[y_train == label])))

  dic={}
  c_prob = class_prob(y_train)

  all_word = []
  for c_word in c_words:
    all_word += c_word
  all_word = np.unique(all_word)
  len_all_word = len(all_word)

  for word in all_word:
    for i, c_word in enumerate(c_words):
      dic[(word, labels[i])] = (len([1 for x in c_word if x == word]) + 1)/(len(c_word)+len_all_word)

  np.save('dic.npy', dic)

  return dic

dic = conditional_prob(X_train, y_train)

In [ ]:
dic

{('+', 'ai'): 0.00021782537665638048,
 ('+', 'sport'): 5.0684237202230105e-05,
 ('+⁄', 'ai'): 3.630422944273008e-05,
 ('+⁄', 'sport'): 7.602635580334515e-05,
 ('aaai', 'ai'): 0.00010891268832819024,
 ('aaai', 'sport'): 2.5342118601115053e-05,
 ('aarau', 'ai'): 3.630422944273008e-05,
 ('aarau', 'sport'): 5.0684237202230105e-05,
 ('aaronson', 'ai'): 7.260845888546016e-05,
 ('aaronson', 'sport'): 2.5342118601115053e-05,
 ('aau', 'ai'): 3.630422944273008e-05,
 ('aau', 'sport'): 5.0684237202230105e-05,
 ('abanba', 'ai'): 3.630422944273008e-05,
 ('abanba', 'sport'): 5.0684237202230105e-05,
 ('abandon', 'ai'): 7.260845888546016e-05,
 ('abandon', 'sport'): 7.602635580334515e-05,
 ('abba', 'ai'): 3.630422944273008e-05,
 ('abba', 'sport'): 5.0684237202230105e-05,
 ('abbrevi', 'ai'): 3.630422944273008e-05,
 ('abbrevi', 'sport'): 5.0684237202230105e-05,
 ('abdel', 'ai'): 3.630422944273008e-05,
 ('abdel', 'sport'): 5.0684237202230105e-05,
 ('abduljabbar', 'ai'): 3.630422944273008e-05,
 ('abduljabba

In [ ]:
def predict(doc, labels, c_prob, dic_name= 'dic.npy'):
  dic = np.load(dic_name, allow_pickle='TRUE').item()

  doc = text_preprocessing(doc)
  tokens = getTokens(doc)
  prob = 1
  for token in tokens:
    if (token, labels[0]) in dic.keys():
      prob *= dic[(token, labels[0])]/dic[(token, labels[1])]

  return prob*(c_prob[0]/c_prob[1])

In [ ]:
#Tensorflow Doc
if predict(X_test[0], ['ai', 'sport'], probs, 'dic.npy') > 1:
  print("ai")
else:
  print("sport")

ai


In [ ]:
#Messi Doc
if predict(X_test[1], ['ai', 'sport'], probs, 'dic.npy') > 1:
  print("ai")
else:
  print("sport")

sport
